# Lab 2

In [105]:
import numpy as np

## Part 1

• Write a program to implement Needleman-Wunsch for proteins

• Check that it is working using HEAGAWGHEE versus PAWHEAE

• Compare to slide 60 in Lecture 09 - Sequence matching.

• Match the protein sequences using your implementation
    SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL with PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY

In [106]:
blosum50 = np.genfromtxt("blosum50.txt", dtype=int)

def blosum_lookup(prot1,prot2):
  amino_acids = dict(zip(['-','A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V'], [i for i in range(len(blosum50))]))
  return blosum50[amino_acids[prot1], amino_acids[prot2]]

In [110]:
def needleman_wunsch(str1, str2):
    cost = -8
    n = len(str1)
    m = len(str2)
    score_matrix = np.zeros((n+1,m+1))
    direction_matrix = np.zeros((n+1,m+1))

    # 1 - mismatch (vertical)
    # 2 - indel (horizontal)
    # 3 - match (diagonal)

    for i in range(1, n+1):
        score_matrix[i,0] = score_matrix[i-1,0] + cost
        direction_matrix[i,0] = 1
    for j in range(1, m+1):
        score_matrix[0,j] = score_matrix[0,j-1] + cost
        direction_matrix[0,j] = 2

    for i in range(1, n+1):
        for j in range(1, m+1):
            blosumScore = blosum_lookup(str1[i-1], str2[j-1])
            match = score_matrix[i-1,j-1] + blosumScore
            mismatch = score_matrix[i-1,j] + cost
            indel = score_matrix[i,j-1] + cost
            
            score = max(match, mismatch, indel)
            score_matrix[i,j] = score

            if score == match:
                direction_matrix[i,j] = 3
            elif score == mismatch:
                direction_matrix[i,j] = 1
            else:
                direction_matrix[i,j] = 2

    print(score_matrix)
    print(direction_matrix)

    alignment1 = ''
    alignment2 = ''

    while n > 0 or m > 0:
        if direction_matrix[n,m] == 3:
            alignment1 = str1[n-1] + alignment1
            alignment2 = str2[m-1] + alignment2
            n = n - 1
            m = m - 1
        elif direction_matrix[n,m] == 2:
            alignment1 = '-' + alignment1
            alignment2 = str2[m-1] + alignment2
            m = m - 1
        else:
            alignment1 = str1[n-1] + alignment1
            alignment2 = '-' + alignment2
            n = n - 1

    print(alignment1)
    print(alignment2)

In [108]:
needleman_wunsch("PAWHEAE","HEAGAWGHEE")

[[  0.  -8. -16. -24. -32. -40. -48. -56. -64. -72. -80.]
 [ -8.  -2.  -9. -17. -25. -33. -41. -49. -57. -65. -73.]
 [-16. -10.  -3.  -4. -12. -20. -28. -36. -44. -52. -60.]
 [-24. -18. -11.  -6.  -7. -15.  -5. -13. -21. -29. -37.]
 [-32. -14. -18. -13.  -8.  -9. -13.  -7.  -3. -11. -19.]
 [-40. -22.  -8. -16. -16.  -9. -12. -15.  -7.   3.  -5.]
 [-48. -30. -16.  -3. -11. -11. -12. -12. -15.  -5.   2.]
 [-56. -38. -24. -11.  -6. -12. -14. -15. -12.  -9.   1.]]
[[0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 3. 3. 3. 2. 3. 2. 2. 2. 3. 3.]
 [1. 3. 3. 3. 2. 3. 2. 2. 2. 2. 2.]
 [1. 1. 1. 3. 3. 3. 3. 2. 2. 2. 2.]
 [1. 3. 3. 3. 3. 3. 1. 3. 3. 2. 2.]
 [1. 1. 3. 2. 3. 3. 3. 1. 3. 3. 3.]
 [1. 1. 1. 3. 2. 3. 3. 3. 1. 1. 3.]
 [1. 1. 3. 1. 3. 3. 3. 3. 3. 3. 3.]]
--P-AW-HEAE
HEAGAWGHE-E


In [109]:
needleman_wunsch("SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL", "PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY")

[[   0.   -8.  -16. ... -256. -264. -272.]
 [  -8.   -1.   -3. ... -243. -251. -259.]
 [ -16.   -9.    0. ... -230. -238. -246.]
 ...
 [-248. -230. -217. ...    5.   -3.  -11.]
 [-256. -238. -225. ...   10.    3.   -5.]
 [-264. -246. -233. ...    2.   15.    7.]]
[[0. 2. 2. ... 2. 2. 2.]
 [1. 3. 3. ... 2. 2. 2.]
 [1. 3. 3. ... 2. 2. 2.]
 ...
 [1. 1. 3. ... 3. 2. 2.]
 [1. 1. 1. ... 3. 3. 3.]
 [1. 1. 1. ... 1. 3. 2.]]
-SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL-
PSPTMEAVTSVEA-STASHPHSTSSYFATTYYHLY
